# Classify Single Target

This notebook calculates the classification coordinate for a single light curve.


In [2]:
import sys

sys.path.insert(0, '../')

import sncosmo
from matplotlib import pyplot as plt
from sndata.csp import dr3
from sndata.sdss import sako18

from analysis_pipeline.models import register_sources
from analysis_pipeline.utils import split_data
from analysis_pipeline.lc_fitting import calc_chisq as _calc_chisq

dr3.download_module_data()
dr3.register_filters(force=True)
register_sources(force=True)


In [ ]:
def calc_chisq(data, model, vparam=4):
    """Calculate the chi-squared
    
    Args:
        data  (Table): Table of light curve data for SNCosmo
        model (Model): SNCosmo model already fitted to the data
        vparam  (int): Number of parameters used in the fit
    """

    chisq, points = _calc_chisq(data, model)
    dof = points - vparam
    return chisq / dof


def fit_for_chisq(data, model, bounds):
    """Fit data with a model and return the chi-squared
    
    Args:
        data  (Table): Table of light curve data for SNCosmo
        model (Model): SNCosmo model
        bounds (dict): bounds for the fit
    
    Returns:
        The reduced chi-squared
    """

    model.set(z=data.meta['redshift'])

    result, fitted_model = sncosmo.fit_lc(data, model, model.param_names[1:], bounds=bounds)
    chisq = calc_chisq(data, fitted_model)
    
    sncosmo.plot_lc(data, fitted_model)
    plt.show()
    
    return chisq

def get_classification_coords(data, bounds, band_names, lambda_eff):
    """Determine the classification coordinates for a given target
    
    Args:
        data             (Table): Table of light curve data for SNCosmo
        bounds            (dict): Bounds for the fit
        band_names   (list[str]): Band names of the parent survey
        lambda_eff (list[float]): Effective wavelength for each band
    
    Returns:
       The x and y classification coordinates
    """

    blue_data, red_data = split_data(data, band_names, lambda_eff)

    print('91bg')
    model_bg = sncosmo.Model(source=sncosmo.get_source('sn91bg'))
    b_bg = fit_for_chisq(blue_data, model_bg, bounds)
    r_bg = fit_for_chisq(red_data, model_bg, bounds)

    print('salt2')
    model_salt2 = sncosmo.Model('salt2')
    b_s2 = fit_for_chisq(blue_data, model_salt2, bounds)
    r_s2 = fit_for_chisq(red_data, model_salt2, bounds)
    return (b_s2 - b_bg, r_s2 - r_bg)


In the following cell we fit with all available data. The cell after that we ignore the NIR.

In [ ]:
data_table = dr3.get_data_for_id('2006gt', True)
bounds = {
    'z': [0.01, 0.8],
    't0': (1003, 1004), # 06gt
    'x0':[0, 0.02],
    'x1': [0.65, 1.25],
    'c': [0, 1]
}

band_names = dr3.band_names
lambda_eff = dr3.lambda_effective

coord = get_classification_coords(data_table, bounds, band_names, lambda_eff)
print('Classification coords:', coord)


In [ ]:
data_table = dr3.get_data_for_id('2006gt', True)
data_table = data_table[data_table['band'] != 'csp_dr3_Y']
data_table = data_table[data_table['band'] != 'csp_dr3_Ydw']
data_table = data_table[data_table['band'] != 'csp_dr3_J']
data_table = data_table[data_table['band'] != 'csp_dr3_H']

band_names = dr3.band_names
lambda_eff = dr3.lambda_effective

coord = get_classification_coords(data_table, bounds, band_names, lambda_eff)
print('Classification coords:', coord)
